In [ ]:
print('Cloning the repo...')
!git clone https://github.com/beratcmn/cortec.git
%cd ./cortec/
print('\nAvailable files:')
%ls
print('\nDownloading the requirements...')
%pip install -qqq -U -r requirements.txt

Cloning the repo...
Cloning into 'cortec'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 118 (delta 16), reused 34 (delta 9), pack-reused 75
Receiving objects: 100% (118/118), 57.32 MiB | 29.60 MiB/s, done.
Resolving deltas: 100% (26/26), done.
/content/cortec

Available files:
CORTEC.ipynb  questions.csv  requirements.txt  results.md  TODO.md
data/         README.md      results.csv       src/

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.4/422.4 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 

In [ ]:
%pip show pybox

Name: pybox
Version: 0.0.3b1
Summary: A Python library for making common things easier and faster such as creating Chat UIs with Gradio or testing LLMs on predefined questions.
Home-page: https://github.com/beratcmn/pybox
Author: Berat Çimen
Author-email: beratcmn@hotmail.com
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: gradio, pandas
Required-by: 


In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage
from langchain.schema.messages import SystemMessage
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

from pybox.llm import GradioChatWebUI
from pybox.llm import Tester as LLMTester

In [ ]:
import os
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["OPENAI_API_BASE"] = "https://demo-moh-opeanaij-ape.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "a275c1b36f42488eb96add2a955aba48"

In [ ]:
model = AzureChatOpenAI(
    deployment_name="Demo-CORTEC-gpt-4",
    model_name="gpt-4",
    temperature=0
)

In [ ]:
result = model(
    [
        HumanMessage(content="What are some touristic places in Paris?")
    ]
)

print(result.content)

1. Eiffel Tower: The most iconic symbol of Paris, offering panoramic views of the city.

2. Louvre Museum: The world's largest art museum and a historic monument in Paris.

3. Notre-Dame Cathedral: A masterpiece of French Gothic architecture, and one of the largest and most well-known church buildings in the world.

4. Montmartre: Known for its bohemian past and the beautiful Sacré-Cœur Basilica, there are also many unique shops and cafes to explore.

5. Champs-Élysées: One of the most famous streets in the world, leading up to the Arc de Triomphe.

6. Palace of Versailles: A royal château in Versailles, known for its Hall of Mirrors, the jewel-like Royal Opera, and the royal apartments.

7. Seine River: Offers boat tours that provide a unique way to see the city's famous landmarks.

8. Musée d'Orsay: An art museum housed in a former railway station, it holds mainly French art dating from 1848 to 1914.

9. Sainte-Chapelle: Known for its stunning stained glass windows, it's considered a

In [ ]:
embeddings = OpenAIEmbeddings(
    deployment="Demo-CORTEC-text-embedding-ada",
    model="text-embedding-ada-002",
    openai_api_base="https://demo-moh-opeanaij-ape.openai.azure.com/",
    openai_api_type="azure",
    chunk_size=16,
)

In [ ]:
model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction="Generate a representation for this sentence that can be used to retrieve related articles:"
)

In [ ]:
!pwd

/content/cortec


In [ ]:
loader = DirectoryLoader('./data/CORTEC_cleaned/', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
texts = text_splitter.split_documents(documents)

In [ ]:
len(texts)

3428

In [ ]:
persist_directory = 'db'

In [ ]:
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embeddings,
                                 persist_directory=persist_directory)

In [ ]:
question = "Paper No. C2020-15065"

docs = vectordb.similarity_search(question, k=10)

for doc in docs:
  print(doc.metadata['source'])

data/CORTEC_cleaned/Design, Construction, Operation,  Maintenance, and Inspection of  Terminal and Tank Facilities.pdf
data/CORTEC_cleaned/00 CorroLogic AST Sales Playbook-R0.pdf
data/CORTEC_cleaned/R Protection of Tank No. F50-T35 floor using Cortec Corrologic Technology.pdf
data/CORTEC_cleaned/R Protection of Tank No. F50-T35 floor using Cortec Corrologic Technology.pdf
data/CORTEC_cleaned/651_Ed3_with_highlighted_changes_final.pdf
data/CORTEC_cleaned/00 CorroLogic AST Sales Playbook-R0.pdf
data/CORTEC_cleaned/NACE Paper 2013.pdf
data/CORTEC_cleaned/2242.pdf
data/CORTEC_cleaned/2242  Mitigating Soil-Side Corrosion on Crude Oil Tank Bottoms Using Vol....pdf
data/CORTEC_cleaned/Evaluation of Corrosion Protection Provided by VpCI-609 in Sand saturate....pdf


In [ ]:
#vectordb.persist()
#vectordb = None

In [ ]:
#vectordb = Chroma(persist_directory=persist_directory,
#                  embedding_function=embeddings)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 10})

In [ ]:
tool = create_retriever_tool(
    retriever,
    "search_cortec_documents",
    "Searches and returns documents regarding the CORTEC."
)
tools = [tool]

In [ ]:
_system_message = """
You are an AI assistant representing the company CORTEC. CORTEC is a company that specializes in corrosion protection solutions and offers a wide range of products and services for various industries around the globe.

Do your best to answer the questions. You are provided with a tool that has the company's files. Try to use the provided tool as much as possible.

If the user is asking for a specific document or a paper try to find the closest document with the relevant Paper No.
""".strip()

system_message = SystemMessage(content=_system_message)

In [ ]:
agent_executor = create_conversational_retrieval_agent(model, tools, verbose=True, system_message=system_message)

In [ ]:
agent_executor.agent.prompt.messages

[SystemMessage(content="You are an AI assistant representing the company CORTEC. CORTEC is a company that specializes in corrosion protection solutions and offers a wide range of products and services for various industries around the globe.\n\nDo your best to answer the questions. You are provided with a tool that has the company's files. Try to use the provided tool as much as possible. \n\nIf the user is asking for a specific document or a paper try to find the closest document with the relevant Paper No.", additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history'),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='{input}', template_format='f-string', validate_template=True), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
result = agent_executor({"input": "hello"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


In [ ]:
result = agent_executor({"input": "what is Compatibility & Interactions between Cathodic Protection and a Vapor?"})



> Entering new AgentExecutor chain...

Invoking: `search_cortec_documents` with `Compatibility & Interactions between Cathodic Protection and a Vapor`


[Document(page_content='incompatibility and maximize th e benefit of compatibility.  \n \nACKNOWLEDGEMENTS  \n \nThe authors wish to acknowledge the assistance of Andreas Skapoulli at Cortec Middle East and Dr. \nMohamad Nagi at the American University of Dubai for the test facilities.  \n \n \nREFERENCES  \n \n1. C. Pynn, K. Abed “ Compatibility & Interactions between Cathodic Protection and a Vapor Phase \nCorrosion Inhibitor ” CORROSION/17, paper no. 9232  (New Orleans, LA: NACE, 2017)', metadata={'page': 6, 'source': 'data/CORTEC_cleaned/11030.pdf'}), Document(page_content='Compatibility & Interactions between Cathodic Protection and a Vapor \nPhase Corrosion Inhibitor \n \nCalvin R. Pynn  \nAEGIS Technical Systems LLC  \nDubai, United Arab Emirates  \nEmail: Calvin.Pynn@eim.ae  \n \nKhalil Abed  \nCORTEC Middle -East \nDubai, Un

In [ ]:
def get_message(userInput: str):
    return agent_executor({"input":userInput})["output"]

In [ ]:
get_message("summarize paper number C2020-15065")



> Entering new AgentExecutor chain...

Invoking: `search_cortec_documents` with `C2020-15065`


[Document(page_content='2.1 Standards, Codes, Publications, and Specifications  ............................................... 4 4 \n2.2 Other References  ..................................................................................................... 5 \n3 DEFINITIONS  .................................................................................................................. \n4 CORROSION OF ABOVEGR OUND STEEL STORAGE T ANKS  ............................ 5', metadata={'page': 1, 'source': 'data/CORTEC_cleaned/651_Ed3_with_highlighted_changes_final.pdf'}), Document(page_content='publication or comments and questions concerning the procedures under which this publication was developed should be directed in writing to the Director of Standards, American Petroleum Institute, 1220 L Street,  N.W., Washington, D.C. 20005. \nRequests for permission to reproduce or translate all or a

"I'm sorry, but the search results for the paper number C2020-15065 do not provide a comprehensive summary of the document. The available snippets contain data tables and copyright information, but they do not provide a clear context or summary of the paper's content. \n\nIt would be best to access the full document for a complete understanding. Would you like to access this document?"

In [ ]:
tester = LLMTester(bot_callback=get_message, questions_csv_path="./questions.csv", questions=[])

In [ ]:
tester.start()



> Entering new AgentExecutor chain...

Invoking: `search_cortec_documents` with `15065`


[Document(page_content='confidential and/or privileged material. Access to this e-mail by anyone else is unauthorized. If you receive this e-mail in \nerror, please contact the sender immediately or the IT Help Desk by telephone +966 (0)2 4250933 and delete the e-mail \nand any attached files from all computers. If you are not the intended recipient, any disclosure, copying, distribution, or \nany action taken in reliance on this e-mail or omission in reliance on this e-mail, is strictly prohibited. Statement and', metadata={'page': 3, 'source': 'data/CORTEC_cleaned/R Protection of Tank No. F50-T35 floor using Cortec Corrologic Technology.pdf'}), Document(page_content='Date R1 R2 R3 R4 R5 R6 R7 R8 R9 \n28/10/2003 -914 -1019.5 -1173.4 -1016.4 -1108.5 -1160 -1250.2 -937.3 -1211.3\n29/11/2003 -917.3 -1031.4 -1163.4 -1039.4 -1096.5 -1123 -1226.3 -945.1 -1197.9\n10/01/2004 -917.3 -1030.4 -1163.3 -105

In [ ]:
webui = GradioChatWebUI(
    bot_callback=get_message,
    title="CORTEC",
    header="## This is a PoC for CORTEC",
    colab_stay_awake=True
)

webui.start()